In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from scipy import stats
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
import timeit

In [56]:
data = pd.read_csv('C:/Users/Cem/Documents/GitHub/Frequent-Pattern-Mining-and-Clustering/dataset/Shill Bidding Dataset.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6321 entries, 0 to 6320
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Record_ID               6321 non-null   int64  
 1   Auction_ID              6321 non-null   int64  
 2   Bidder_ID               6321 non-null   object 
 3   Bidder_Tendency         6321 non-null   float64
 4   Bidding_Ratio           6321 non-null   float64
 5   Successive_Outbidding   6321 non-null   float64
 6   Last_Bidding            6321 non-null   float64
 7   Auction_Bids            6321 non-null   float64
 8   Starting_Price_Average  6321 non-null   float64
 9   Early_Bidding           6321 non-null   float64
 10  Winning_Ratio           6321 non-null   float64
 11  Auction_Duration        6321 non-null   int64  
 12  Class                   6321 non-null   int64  
dtypes: float64(8), int64(4), object(1)
memory usage: 642.1+ KB


In [57]:
data.head()

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,7,0


In [58]:
data1 = data[['Bidder_Tendency',
       'Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding',
       'Auction_Bids', 'Starting_Price_Average', 'Early_Bidding',
       'Winning_Ratio', 'Auction_Duration', 'Class']]
data_list = data1.values.tolist()
data1.head()

,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
1,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
2,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0
3,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,5,0
4,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,7,0


In [59]:
data1_col_dur_list = ['Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding', 'Auction_Bids', 'Starting_Price_Average', 'Early_Bidding', 'Winning_Ratio']

for col in data1_col_dur_list:
    print(col)
    dur_list = [1,3,5,7,10]

    for i in range(0,4):
        dur_list1 = dur_list[i+1:]

        for j in dur_list1:
            print('Duration ',dur_list[i],'~ Duration ',j, ': pvalue=', stats.kruskal(data1[data1['Auction_Duration']==dur_list[i]][col], data1[data1['Auction_Duration']==j][col])[1])
    
    print('-------------------------------------------------------------------------------------------------------------')


Bidder_Tendency
Duration  1 ~ Duration  3 : pvalue= 0.6966023397831832
Duration  1 ~ Duration  5 : pvalue= 0.4793660843340304
Duration  1 ~ Duration  7 : pvalue= 0.9344002917017548
Duration  1 ~ Duration  10 : pvalue= 0.04517460218654448
Duration  3 ~ Duration  5 : pvalue= 0.7271581404175813
Duration  3 ~ Duration  7 : pvalue= 0.729742480022295
Duration  3 ~ Duration  10 : pvalue= 0.07443894016187297
Duration  5 ~ Duration  7 : pvalue= 0.48010036934088474
Duration  5 ~ Duration  10 : pvalue= 0.1183085861883225
Duration  7 ~ Duration  10 : pvalue= 0.05026859712280161
-------------------------------------------------------------------------------------------------------------
Bidding_Ratio
Duration  1 ~ Duration  3 : pvalue= 0.8239099593171679
Duration  1 ~ Duration  5 : pvalue= 0.1283495308898885
Duration  1 ~ Duration  7 : pvalue= 0.008958584749331476
Duration  1 ~ Duration  10 : pvalue= 0.00017401995356576726
Duration  3 ~ Duration  5 : pvalue= 0.07911828899622692
Duration  3 ~ Durati

In [60]:
data1['Auction_Duration'].value_counts()

7     2427
3     1408
1     1289
5     1060
10     137
Name: Auction_Duration, dtype: int64

In [61]:
data_last =[]

for a in data_list:
    temp =[]
    for b in a:
        tobeadded = int(b/0.5)*-1
        if(tobeadded in temp):
            continue
        temp.append(tobeadded)
    
    data_last.append(temp)  

In [62]:
X1 = data1[data1['Auction_Duration']==1][['Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding', 'Auction_Bids', 'Starting_Price_Average', 'Early_Bidding', 'Winning_Ratio']]
y1 = data1[data1['Auction_Duration']==1]['Class']
X3 = data1[data1['Auction_Duration']==3][['Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding', 'Auction_Bids', 'Starting_Price_Average', 'Early_Bidding', 'Winning_Ratio']]
y3 = data1[data1['Auction_Duration']==3]['Class']
X5 = data1[data1['Auction_Duration']==5][['Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding', 'Auction_Bids', 'Starting_Price_Average', 'Early_Bidding', 'Winning_Ratio']]
y5 = data1[data1['Auction_Duration']==5]['Class']
X7 = data1[data1['Auction_Duration']==7][['Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding', 'Auction_Bids', 'Starting_Price_Average', 'Early_Bidding', 'Winning_Ratio']]
y7 = data1[data1['Auction_Duration']==7]['Class']
X10 = data1[data1['Auction_Duration']==10][['Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding', 'Auction_Bids', 'Starting_Price_Average', 'Early_Bidding', 'Winning_Ratio']]
y10 = data1[data1['Auction_Duration']==10]['Class']

In [63]:
te = TransactionEncoder()
te_ary = te.fit(data_last).transform(data_last)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,-20,-14,-10,-6,-2,-1,0
0,False,False,True,False,False,True,True
1,False,False,True,False,False,True,True
2,False,False,True,False,True,True,True
3,False,False,True,False,True,True,True
4,False,True,False,False,False,True,True
...,...,...,...,...,...,...,...
6316,False,False,False,True,True,True,True
6317,False,True,False,False,False,True,True
6318,False,True,False,False,False,True,True
6319,False,True,False,False,False,True,True


## Apriori

In [68]:
aprioriSup =[]
aprioriConf=[]
aprioriLift=[]
start = timeit.default_timer()
aprioriDF = apriori(df, min_support=0.1, use_colnames=True)
aprioriDF['length'] = aprioriDF['itemsets'].apply(lambda x: len(x))
stop = timeit.default_timer()
runtime1 = stop - start
aprioriRule = association_rules(aprioriDF, min_threshold=0.1)
aprioriRule["antecedents"] = aprioriRule["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
aprioriRule["consequents"] = aprioriRule["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
aprioriList = aprioriRule.values.tolist()

for el in aprioriList:
    aprioriSup.append(el[4])
    aprioriConf.append(el[5])
    aprioriLift.append(el[4])
    
aprioriDF

,support,itemsets,length
0,0.383958,(-14),1
1,0.167695,(-10),1
2,0.222750,(-6),1
3,0.372726,(-2),1
4,0.826293,(-1),1
5,0.998893,(0),1
6,0.316564,"(-14, -1)",2
7,0.383325,"(0, -14)",2
8,0.138427,"(-10, -1)",2
9,0.167537,"(0, -10)",2


In [69]:
start = timeit.default_timer()
aprioriDF[(aprioriDF['length'] > 1)].sort_values(by=['support'], ascending=False)
stop = timeit.default_timer()
print('Runtime of Apriori: ', ((stop - start)+runtime1)/2)
aprioriDF

Runtime of Apriori:  0.006593849999262602


,support,itemsets,length
0,0.383958,(-14),1
1,0.167695,(-10),1
2,0.222750,(-6),1
3,0.372726,(-2),1
4,0.826293,(-1),1
5,0.998893,(0),1
6,0.316564,"(-14, -1)",2
7,0.383325,"(0, -14)",2
8,0.138427,"(-10, -1)",2
9,0.167537,"(0, -10)",2


## FP-Growth

In [70]:
fpgrowthSup =[]
fpgrowthConf=[]
fpgrowthLift=[]
start = timeit.default_timer()
fpgrowthDF=fpgrowth(df, min_support=0.1, use_colnames=True)
fpgrowthDF['length'] = fpgrowthDF['itemsets'].apply(lambda x: len(x))
fpgrowthRule = association_rules(fpgrowthDF, min_threshold=0.1)
fpgrowthRule["antecedents"] = fpgrowthRule["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
fpgrowthRule["consequents"] = fpgrowthRule["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
fpgrowthList = fpgrowthRule.values.tolist()
stop = timeit.default_timer()
print('Runtime of FP-Growth: ', (stop - start))

for el in fpgrowthList:
    fpgrowthSup.append(el[4])
    fpgrowthConf.append(el[5])
    fpgrowthLift.append(el[4])
fpgrowthDF

Runtime of FP-Growth:  0.07905910000044969


,support,itemsets,length
0,0.998893,(0),1
1,0.826293,(-1),1
2,0.167695,(-10),1
3,0.372726,(-2),1
4,0.383958,(-14),1
5,0.222750,(-6),1
6,0.825186,"(0, -1)",2
7,0.167537,"(0, -10)",2
8,0.138427,"(-10, -1)",2
9,0.138269,"(0, -10, -1)",3


## ECLAT

In [53]:
eclatSup =[]
eclatConf=[]
eclatLift=[]
start = timeit.default_timer()
eclatDF = association_rules(aprioriDF, metric="support", min_threshold=0.05)
eclatDF["antecedents"] = eclatDF["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
eclatDF["consequents"] = eclatDF["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
eclatList = eclatDF.values.tolist()
stop = timeit.default_timer()
print('Runtime of ECLAT: ', (stop - start))

for el in eclatList:
    eclatSup.append(el[4])
    eclatConf.append(el[5])
    eclatLift.append(el[4])
eclatDF

Runtime of ECLAT:  0.007058600000164006


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,-14,-2,0.383958,0.372726,0.087486,0.227853,0.611316,-0.055625,0.812377
1,-2,-14,0.372726,0.383958,0.087486,0.234720,0.611316,-0.055625,0.804989
2,-14,-1,0.383958,0.826293,0.316564,0.824475,0.997799,-0.000698,0.989639
3,-1,-14,0.826293,0.383958,0.316564,0.383113,0.997799,-0.000698,0.998630
4,0,-14,0.998893,0.383958,0.383325,0.383750,0.999459,-0.000208,0.999663
...,...,...,...,...,...,...,...,...,...
65,-2,0,0.305806,0.383325,0.072457,0.236937,0.618110,-0.044766,0.808157
66,0,-14,0.998893,0.073090,0.072457,0.072537,0.992441,-0.000552,0.999404
67,-14,0,0.383958,0.304699,0.072457,0.188710,0.619334,-0.044535,0.857032
68,-2,0,0.372726,0.315931,0.072457,0.194397,0.615316,-0.045299,0.849139


In [54]:
print("Support Mean of Apriori",sum(aprioriSup) / len(aprioriSup))
print("Support Mean of FP Growth",sum(fpgrowthSup) / len(fpgrowthSup))
print("Support Mean of ECLAT",sum(eclatSup) / len(eclatSup))

print("Confidence Mean of Apriori",sum(aprioriConf) / len(aprioriConf))
print("Confidence Mean of FP Growth",sum(fpgrowthConf) / len(fpgrowthConf))
print("Confidence Mean of ECLAT",sum(eclatConf) / len(eclatConf))

print("Lift Mean of Apriori",sum(aprioriLift) / len(aprioriLift))
print("Lift Mean of FP Growth",sum(fpgrowthLift) / len(fpgrowthLift))
print("Lift Mean of ECLAT",sum(eclatLift) / len(eclatLift))

Support Mean of Apriori 0.19450358216376243
Support Mean of FP Growth 0.19450358216376248
Support Mean of ECLAT 0.19450358216376243
Confidence Mean of Apriori 0.4858276389763707
Confidence Mean of FP Growth 0.4858276389763707
Confidence Mean of ECLAT 0.4858276389763707
Lift Mean of Apriori 0.19450358216376243
Lift Mean of FP Growth 0.19450358216376248
Lift Mean of ECLAT 0.19450358216376243
